In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.metrics import F1Score, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Thu Apr 25 06:15:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
train_dir = '/content/drive/MyDrive/x-ray-data/chest_xray_refined/train'
test_dir = '/content/drive/MyDrive/x-ray-data/chest_xray_refined/test'
val_dir = '/content/drive/MyDrive/x-ray-data/chest_xray_refined/val'

img_gen = ImageDataGenerator(rescale=1./255)

train_generator = img_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

test_generator = img_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

val_generator = img_gen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
resnet50_base_model = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
def build_model(base_model):
  for layer in base_model.layers:
    layer.trainable=False

  x = base_model.output
  x = GlobalMaxPooling2D()(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(256, activation='relu')(x)
  x = Dense(128, activation='relu')(x)

  outputs = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=base_model.input, outputs=outputs)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(thresholds=0.5), Recall(thresholds=0.5), F1Score(threshold=0.5)])

  return model

In [ ]:
model = build_model(resnet50_base_model)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                            

# Training

In [ ]:
checkpoint_path = "/content/drive/MyDrive/xray-model-office-3-layers/cp-{epoch:04d}.ckpt"

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=815)

In [ ]:
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=val_generator,
                    callbacks=[EarlyStopping(patience=10), cp_callback],
                    batch_size=32)

Epoch 1/50
163/163 [==============================] - 84s 477ms/step - loss: 0.4371 - accuracy: 0.9135 - precision_1: 0.9426 - recall_1: 0.9409 - f1_score: 0.9418 - val_loss: 0.3189 - val_accuracy: 0.8750 - val_precision_1: 0.8000 - val_recall_1: 1.0000 - val_f1_score: 0.8889
Epoch 2/50
163/163 [==============================] - 75s 457ms/step - loss: 0.1079 - accuracy: 0.9613 - precision_1: 0.9744 - recall_1: 0.9734 - f1_score: 0.9739 - val_loss: 0.2116 - val_accuracy: 0.9375 - val_precision_1: 0.8889 - val_recall_1: 1.0000 - val_f1_score: 0.9412
Epoch 3/50
163/163 [==============================] - 74s 456ms/step - loss: 0.0659 - accuracy: 0.9770 - precision_1: 0.9835 - recall_1: 0.9855 - f1_score: 0.9845 - val_loss: 0.0482 - val_accuracy: 1.0000 - val_precision_1: 1.0000 - val_recall_1: 1.0000 - val_f1_score: 1.0000
Epoch 4/50
163/163 [==============================] - 75s 462ms/step - loss: 0.0633 - accuracy: 0.9764 - precision_1: 0.9838 - recall_1: 0.9845 - f1_score: 0.9841 - val_

In [ ]:
model.save('/content/drive/MyDrive/3-layers-office-resnetXray-weights.h')

In [ ]:
res1 = model.evaluate(train_generator)
res2 = model.evaluate(test_generator)
res3 = model.evaluate(val_generator)

1/1 [==============================] - 0s 398ms/step - loss: 0.2180 - accuracy: 0.9375 - precision_1: 0.8889 - recall_1: 1.0000 - f1_score: 0.9412


In [ ]:
res1

[5.720099215977825e-05, 1.0, 1.0, 1.0, array([1.], dtype=float32)]

In [ ]:
res2

[2.0928850173950195,
 0.8125,
 0.772455096244812,
 0.9923076629638672,
 array([0.86868685], dtype=float32)]

In [ ]:
res3

[0.21802139282226562,
 0.9375,
 0.8888888955116272,
 1.0,
 array([0.94117653], dtype=float32)]

In [ ]:
import pickle

with open('/content/drive/MyDrive/history-office-3-layers.pkl', "wb") as file:
    pickle.dump(history, file)